## Download the data

In [1]:
import numpy as np
import os
import tensorflow as tf

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]
###### Do not modify here ###### 

# print(tf.argmax(mnist.train.labels))


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


## Get the data shape

In [2]:
print('Train', mnist.train.num_examples, 
      'Validation', mnist.validation.num_examples,
      'Test', mnist.test.num_examples)

print('Train images :', X_train1.shape,
      'Labels :', y_train1.shape)

print('Validation images :', X_valid1.shape,
      'Labels :', y_valid1.shape)

print('Test images :', X_test1.shape,
      'Labels :', y_test1.shape)

Train 55000 Validation 5000 Test 10000
Train images : (28038, 784) Labels : (28038,)
Validation images : (2558, 784) Labels : (2558,)
Test images : (5139, 784) Labels : (5139,)


## Make data to one hot encoding

In [3]:
# y_train1 = np.eye(5)[y_train1.reshape(-1)]
# y_valid1 = np.eye(5)[y_valid1.reshape(-1)]
# y_test1 = np.eye(5)[y_test1.reshape(-1)]

# print('Train', y_train1.shape, 
#       'Valid', y_valid1.shape,
#       'Test', y_test1.shape)
y_train1 = y_train1.reshape(-1, 1)
y_valid1 = y_valid1.reshape(-1, 1)
y_test1 = y_test1.reshape(-1, 1)


## Construct the layer

In [4]:
def layer(output_dim, input_dim, inputs, activation=None):
    W = tf.Variable(tf.random_normal([input_dim, output_dim]))
    b = tf.Variable(tf.random_normal([1, output_dim]))
    XWb = tf.matmul(inputs, W) + b
    if activation is None:
        outputs = XWb
    else:
        outputs = activation(XWb)
    return outputs

### Contruct the input layer

In [5]:
x = tf.placeholder("float", [None, 784])

### Contruct the hidden layer

In [6]:
h1 = layer(output_dim = 128, input_dim = 784, inputs = x, activation = tf.nn.elu)
h2 = layer(output_dim = 128, input_dim = 128, inputs = h1, activation = tf.nn.elu)
h3 = layer(output_dim = 128, input_dim = 128, inputs = h2, activation = tf.nn.elu)
h4 = layer(output_dim = 128, input_dim = 128, inputs = h3, activation = tf.nn.elu)
h5 = layer(output_dim = 128, input_dim = 128, inputs = h4, activation = tf.nn.elu)

### contruct the output layer

In [7]:
y_predict = layer(output_dim = 1, input_dim = 128, inputs = h5, activation = None)

In [8]:
y_label = tf.placeholder("float", [None, 1])

In [9]:
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_predict, labels=y_label))
# spec use tf.nn.sparse_softmax_cross_entropy_with_logits

In [10]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001) \
                    .minimize(loss_function)

In [11]:
correct_prediction = tf.equal(y_label, y_predict)

In [12]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [13]:
from time import time
startTime = time()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [14]:
sess.run(optimizer, feed_dict={x: X_train1, y_label: y_train1})        

In [16]:
loss, acc = sess.run([loss_function, accuracy], 
                     feed_dict={x: X_valid1, y_label: y_valid1})

In [17]:
loss

0.0

In [18]:
acc

0.0